## Import

In [1]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [1]:
import wandb
wandb.login()
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="face-recognition",
    
    # track hyperparameters and run metadata
    config={
    'LEARNING_RATE':3e-4,
    'IMG_SIZE':224,
    "architecture": "Efficientnet-b0",
    'EPOCHS':100,
    'SEED':42,
    'BATCH_SIZE':32,
    "dataset": "kaggle_dataset_celeribrity_faces",
    }# hyperparameter
)

# hyperparameter
CFG = wandb.config
CFG

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anthoniy. Use `wandb login --relogin` to force relogin


{'LEARNING_RATE': 0.0003, 'IMG_SIZE': 224, 'architecture': 'Efficientnet-b0', 'EPOCHS': 100, 'SEED': 42, 'BATCH_SIZE': 32, 'dataset': 'kaggle_dataset_celeribrity_faces'}

In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torch.autograd import Variable


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing

import warnings
warnings.filterwarnings(action='ignore') 

c:\Users\DoSungjin\anaconda3\envs\gpu-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

os.chdir('../DATA')
os.getcwd()


'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [4]:
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

## Hyperparameter Setting

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [43]:
all_img_list = glob.glob('/facedataset/*/*/*')
# 데이터셋 디렉토리 경로
dataset_dir = "facedataset/Face Data\\Face Dataset"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    if not item.startswith('.'):
        item_path = os.path.join(dataset_dir, item)
        for file in os.listdir(item_path):
            if not file.startswith('.'):
                all_img_list.append(os.path.join(item_path, file))

                folder_list.append(item)

In [44]:
len(folder_list)

8441

In [45]:
all_img_list[0].split('\\')[-1]

'0.jpg'

In [46]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x: int(str(x).split('\\')[-2]))

df.head()

,img_path,label
0,facedataset/Face Data\Face Dataset\0\0.jpg,0
1,facedataset/Face Data\Face Dataset\0\1.jpg,0
2,facedataset/Face Data\Face Dataset\0\2.jpg,0
3,facedataset/Face Data\Face Dataset\0\3.jpg,0
4,facedataset/Face Data\Face Dataset\1\0.jpg,1


In [47]:
df['label'].value_counts()

label
7777    237
856      50
1467     50
531      50
536      50
       ... 
479       2
48        2
481       2
482       2
242       2
Name: count, Length: 1681, dtype: int64

In [49]:
df[df['label']==7777]

,img_path,label
6915,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
6916,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
6917,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
6918,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
6919,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
...,...,...
7147,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
7148,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
7149,facedataset/Face Data\Face Dataset\7777\Leonar...,7777
7150,facedataset/Face Data\Face Dataset\7777\Leonar...,7777


In [10]:
# # train val split
# train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])


## Label-Encoding

In [11]:
# le = preprocessing.LabelEncoder()
# train['label'] = le.fit_transform(train['label'])
# val['label'] = le.transform(val['label'])


In [12]:
# label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print("Label mapping:")
# print(label_mapping)
# # train_folder = 'train'
# # # 작업 디렉토리 변경
# # os.chdir(train_folder)

# # 폴더명 변경
# for old_name, new_label in label_mapping.items():
#     os.rename(old_name, str(new_label))

In [10]:
import json

# Load the label mapping from the JSON file
with open('label_mapping.json', 'r') as f:
    label_mapping = json.load(f)

# Convert the values to a list
values_list = list(label_mapping.values())

# Get the keys that need to be modified
keys_to_update = []
for key, value in label_mapping.items():
    if value in values_list[:18]:
        keys_to_update.append(key)

# Update the values of the keys
for key in keys_to_update:
    label_mapping[key] = values_list.index(label_mapping[key]) + 1


In [11]:
new_label_mapping = {key: idx for idx, key in enumerate(label_mapping.keys())}
label_mapping = new_label_mapping


## CustomDataset

In [12]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [13]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
                           
test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [17]:
# #weighted sampling
# a_list = df['label'].value_counts()
# class_counts = df['label'].value_counts().to_list()
# class_weights_dict = {label: 1/count for label, count in a_list.items()}

# labels_lists = df['label'].to_list()
# weights = [class_weights_dict[label] for label in labels_lists]
# num_samples = sum(class_counts)
# sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)
# sampler

In [18]:
# # make dataloader
# train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
# train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)

# val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [14]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha] * 19)
        self.alpha[18] = 1 - alpha
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C

        if not isinstance(target, torch.Tensor):
            target = torch.tensor(target, dtype=torch.long)

        target = target.view(-1, 1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()


In [18]:
df = df_oversampled.copy()

## Model Define

In [19]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(df['label'].unique()), freeze_backbone=True):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
        if freeze_backbone:
            for param in self.backbone.parameters():
                param.requires_grad = False
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [20]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    
    return preds

In [23]:
from sklearn.model_selection import StratifiedKFold
import gc
from datetime import datetime, timezone, timedelta

X = df['img_path']
y = df['label']
criterion = FocalLoss(gamma=2, alpha=0.25)
# Define the number of folds and other parameters
n_splits = 5
seed = 42

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = torch.tensor(labels, dtype=torch.int64).to(device)

            output = model(imgs)
            loss = criterion(output, labels)

            val_loss.append(loss.item())

    val_loss = np.mean(val_loss)
    return val_loss

def trains(model, optimizer, train_loader, val_loader, scheduler, device, freeze_backbone=True, patience=6):
    model.to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)

    best_loss = float('inf')
    best_model = None

    if freeze_backbone:
        for param in model.backbone.parameters():
            param.requires_grad = False

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = torch.tensor(labels, dtype=torch.int64).to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            _val_loss = validation(model, criterion, val_loader, device)
            _train_loss = np.mean(train_loss)
            print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

            if scheduler is not None:
                scheduler.step(_val_loss)

            if _val_loss < best_loss:
                best_loss = _val_loss
                best_model = model.state_dict().copy()
                torch.save(best_model, "best_model.pth")
                early_stopping_counter = 0  # Reset the counter
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print("Early stopping triggered! No improvement in validation loss.")
                    break

    return best_model


def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)

            pred = model(imgs)

            preds += pred.argmax(1).detach().cpu().numpy().tolist()

    return preds

infer_model = BaseModel()
fold_results = []  # Store results for each fold
preds_list = []  # Store predictions for each fold
true_labels = []
test = pd.read_csv('test.csv')
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("Fold:", fold)

    train_X, val_X = X[train_index], X[val_index]
    train_y, val_y = y[train_index], y[val_index]

    # Create train and validation datasets
    train_dataset = CustomDataset(train_X.values, train_y.values, train_transform)
    val_dataset = CustomDataset(val_X.values, val_y.values, test_transform)

    # Create train and validation data loaders
    train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    # Define your deep learning model
    model = BaseModel(freeze_backbone=False)  # Unfreeze the backbone for fine-tuning
    model.eval()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=6, threshold_mode='abs', min_lr=1e-8, verbose=True)

    # Train your model
    best_model_weights = trains(model, optimizer, train_loader, val_loader, scheduler, device, freeze_backbone=False)

    # Load the best model weights
    model.load_state_dict(best_model_weights)



    # Evaluate the model on the validation set
    val_loss = validation(model, criterion, val_loader, device)
    print("Validation Loss:", val_loss)

    fold_results.append(val_loss)  # Store the result for this fold

    # Save the best model
    torch.save(model.state_dict(), f"model_fold{fold}.pth")
    infer_model.load_state_dict(torch.load(f"model_fold{fold}.pth"))
    infer_model.to(device)
    preds = inference(infer_model, test_loader, device)
    # Store the predictions for this fold
    preds_list.append(preds)
    # Log metrics to Wandb
    wandb.log({"Fold": fold, "Validation Loss": val_loss})

    # Clear unnecessary memory
    del train_dataset, val_dataset, train_loader, val_loader, model, optimizer, scheduler
    torch.cuda.empty_cache()
    gc.collect()

# Compute aggregate statistics
avg_val_loss = np.mean(fold_results)
std_val_loss = np.std(fold_results)
# Log aggregate metrics to Wandb
wandb.log({"Average Validation Loss": avg_val_loss, "Standard Deviation of Validation Loss": std_val_loss})

# Finish Wandb run
wandb.finish()



Fold: 0


  1%|          | 1/144 [00:28<1:07:06, 28.16s/it]

Epoch [1], Train Loss : [0.73025] Val Loss : [0.86513]


  1%|▏         | 2/144 [00:39<43:02, 18.19s/it]  

Epoch [1], Train Loss : [0.77901] Val Loss : [0.70116]


...

Epoch [100], Train Loss : [0.32092] Val Loss : [0.30634]
Early stopping triggered! No improvement in validation loss.


100%|██████████| 36/36 [00:16<00:00,  2.20it/s]


Validation Loss: 0.30634064185950494


100%|██████████| 25/25 [00:11<00:00,  2.18it/s]


Average Validation Loss,▁
Fold,▁▃▅▆█
Standard Deviation of Validation Loss,▁
Validation Loss,█▂▂▂▁
Average Validation Loss,0.33735
Fold,4
Standard Deviation of Validation Loss,0.04163
Validation Loss,0.30634


In [22]:
from scipy.stats import mode

# Perform ensemble by taking the majority vote
ensemble_preds = mode(preds_list, axis=0)[0].flatten()


new_preds = []
for pred in ensemble_preds:
    for key, value in label_mapping.items():
        if value == pred:
            new_preds.append(key)
            break

submit = pd.read_csv('sample_submission.csv')
submit['label'] = new_preds

# Save the submission file
kst = timezone(timedelta(hours=9))
train_serial = datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('../result', train_serial)
os.makedirs(Record_path, exist_ok=True)

submit.to_csv(os.path.join(Record_path, 'submit.csv'), index=False)


ValueError: Length of values (0) does not match length of index (792)

In [ ]:
from collections import Counter
Counter(submit.label)

Counter({'훼손': 605,
         '꼬임': 63,
         '걸레받이수정': 45,
         '오염': 28,
         '곰팡이': 14,
         '오타공': 15,
         '몰딩수정': 4,
         '터짐': 11,
         '면불량': 4,
         '피스': 1,
         '들뜸': 2})

In [23]:
list_dir = os.listdir('../result')
for x in list_dir:
    if not x.endswith('csv'):  
        submit_path = os.path.join('../result', x, 'submit.csv')
        submit = pd.read_csv(submit_path)
        print(len(submit.label.unique()), x)

17 20230516_121017
17 20230516_222825
16 20230518_232630
16 20230519_004356
16 20230519_012220
17 20230519_031010
16 20230519_124821
7 20230519_143519
11 20230519_150640
11 20230519_155612
12 20230519_164318
13 20230520_225922
11 20230521_174835
